In [14]:
# Import libraries
import numpy as np
import pandas as pd
import string
from datetime import datetime
import matplotlib.pyplot as plt

In [15]:
# Input parameters
export_feedback_table = False
inputfile_sssummary  = 'output/studentsummary2_20240222.csv'
inputfile_assignment = 'rawdata/assignment_20240222.csv'
inputfile_feedback   = 'rawdata/feedback_20240222.csv'
outputfile_feedback  = 'output/feedback_pretest_20240222.xlsx'

In [16]:
# Import raw data and converting 'created' to datetime
df_sssummary  = pd.read_csv(inputfile_sssummary, index_col=0)
df_assignment = pd.read_csv(inputfile_assignment, index_col=0)
df_feedback   = pd.read_csv(inputfile_feedback, index_col=0)
df_sssummary['1st_created'] = pd.to_datetime(df_sssummary['1st_created'])
df_assignment['created']    = pd.to_datetime(df_assignment['created'])
df_feedback['created']      = pd.to_datetime(df_feedback['created'])

In [17]:
df_sssummary

,anonym,1st_assignment_id,1st_instance_id,1st_created,1st_finished,2nd_assignment_id,2nd_instance_id,2nd_created,2nd_finished
0,AS23S0335,9cc8536e-35ac-4874-807d-9da3d47a4a89,SLF6A,2023-11-02 08:53:20.875,2023-11-02 09:25:30.675000,45209efe-cda7-480f-8984-1a3f780c840d,MYF6A,2023-12-14 07:11:24.502000,2023-12-14 07:34:11.516000
1,AS23S0336,773fac84-9653-4706-94f2-555506fc3c33,MLF6A,2023-11-02 08:58:00.657,2023-11-02 09:22:16.908000,d2fa8b28-d4d6-4c19-82aa-da16bd7a5788,SYF6A,2023-12-14 07:05:41.840000,2023-12-14 07:35:03.174000
2,AS23S0338,7bc2eab6-f284-42ae-b22b-5edfb37dbe8b,SPF6A,2023-11-02 08:54:09.316,2023-11-02 09:38:22.077000,af7ce587-f8ad-4459-93b0-57fd57d9f51a,MTF6A,2023-12-14 07:08:11.195000,2023-12-14 07:51:40.348000
3,AS23S0339,7448ecde-72d7-4a88-817f-ed4a726879b9,MPF6A,2023-11-02 08:58:04.547,2023-11-02 09:22:15.866000,10119783-d851-4169-a3cd-cd90e31b3c4d,SYF6A,2023-12-14 07:11:25.845000,2023-12-14 07:50:46.121000
4,AS23S0340,d47c0ae8-7835-460b-95fa-c7628e5b2a6f,SLF6A,2023-11-02 08:54:17.666,2023-11-02 09:38:28.370000,4ba95ff5-dce4-4fb3-9ede-5f8b1e85aaa3,MTF6A,2023-12-14 07:10:13.541000,2023-12-14 07:48:41.943000
...,...,...,...,...,...,...,...,...,...
164,KS23S7504,1daffd53-64f0-423f-a501-7aff26dd1de0,SLU5A,2023-11-03 07:28:42.896,2023-11-03 07:50:57.367000,NaN,NaN,NaN,NaN
165,KS23S7505,88363ab5-e685-47fb-a086-47298f636902,SLU5A,2023-11-03 07:29:29.346,NaN,NaN,NaN,NaN,NaN
166,KS23S7506,0041f35d-0cbd-4efe-82c4-b62a72aec1d4,MLU5A,2023-11-03 07:28:27.723,2023-11-03 08:00:36.275000,NaN,NaN,NaN,NaN
167,KS23S7507,43da8763-68ef-4ed9-8735-162eeceedf07,SPU5A,2023-11-03 07:28:50.540,2023-11-03 08:00:26.558000,NaN,NaN,NaN,NaN


In [32]:
# Define the target group assignment_id for analysis

# Choose Experimental Group, Control Group or Both
df_target = df_sssummary[df_sssummary['anonym'].str.startswith("A")]   # Experimental Group
#df_target = df_sssummary[df_sssummary['anonym'].str.startswith("K")]   # Control Group
#df_target = df_sssummary                                               # Both Group

# Choose whether filtered rows with both tests
#df_target = df_target.dropna()
df_target = df_target[df_target.isna().any(axis=1)]

# Choose pre-test, post-test or Both
target_assid = df_target['1st_assignment_id'].tolist()   # Pre-test, Expt Group
#target_assid = df_target['2nd_assignment_id'].tolist()   # Post-test, Expt Group
#target_assid = df_target['1st_assignment_id'].tolist() + df_sssummary_bothtests['2nd_assignment_id'].tolist()   # Both tests, Expt Group

print(len(target_assid))
df_target

8


,anonym,1st_assignment_id,1st_instance_id,1st_created,1st_finished,2nd_assignment_id,2nd_instance_id,2nd_created,2nd_finished
88,AS23S0455,415a7c0c-fc1e-4618-b1d0-5b9de74ea63f,SLT5A,2023-11-07 07:26:41.934,NaN,81c6d13c-095e-4dde-8689-c68a99df7f07,MYT5A,2023-12-12 07:15:21.616000,2023-12-12 07:39:40.162000
89,AS23S0456,b0774673-c79c-4d68-adcf-8aca56a44e5f,SPT5A,2023-11-07 07:27:33.952,2023-11-07 08:09:43.651000,7017b031-cd99-4bd4-8031-06b23967829e,MTT5A,2023-12-12 07:13:05.644000,NaN
90,AS23S6364,NaN,NaN,NaT,NaN,0df48027-0050-4aee-8784-2a43248438f1,SYF6B,2023-12-14 08:09:29.342000,2023-12-14 08:49:25.350000
91,AS23S6407,NaN,NaN,NaT,NaN,38665b25-5db6-41a0-a237-aa12832b282c,SYF5A,2023-12-14 09:11:42.280000,2023-12-14 09:48:42.418000
92,AS23S6435,NaN,NaN,NaT,NaN,30f080b2-96e4-4e5f-8fbd-efa9f211aaf2,STP6B,2023-12-20 08:18:33.202000,2023-12-20 08:45:30.337000
93,AS23S7350,904cb301-bf6e-47d0-bc3c-7c32e6939d4c,SPF6B,2023-11-02 10:42:41.917,2023-11-02 11:22:38.206000,NaN,NaN,NaN,NaN
94,AS23S7394,911e7cad-2261-4755-825d-dd734c8bfaa1,SPF5A,2023-11-02 07:24:41.273,2023-11-02 08:07:14.276000,NaN,NaN,NaN,NaN
95,AS23S7403,69a829b7-c128-46d8-9d04-c5a50b8623c7,MPF5A,2023-11-02 07:25:57.118,2023-11-02 08:07:18.270000,NaN,NaN,NaN,NaN


In [19]:
# Add the 'time_elapsed' column

def calculate_time_difference(row, df_a):
    sid = row['assignment_id'] 
    action_time = row['created']
    # Check if student_id exists in df_a
    if sid in df_a.index:
        starting_time = df_a.at[sid, 'created']
        return (action_time - starting_time).total_seconds()
    else:
        return None  # or some other default value

df_feedback['time_elapsed'] = df_feedback.apply(lambda row: calculate_time_difference(row, df_assignment), axis=1)

In [20]:
# Sort the DataFrame by 'assignment_id' and 'created' to ensure the order
df_feedback = df_feedback.sort_values(by=['assignment_id', 'created'])

# Add a new column 'sequence' which enumerates each student's rows starting from 1
df_feedback['count'] = df_feedback.groupby('assignment_id').cumcount() + 1

In [21]:
# Filtered the feedback df with the target assignment_id
df_feedback_filtered = df_feedback[df_feedback['assignment_id'].isin(target_assid)]
df_feedback_filtered

,assignment_id,phase,type_id,feedback_text,details,created,time_elapsed,count
action_id,,,,,,,,
666ae044-faf5-4564-b41d-624c578f4f52,03d9d713-81e4-4b57-b9f0-80397d5e2fca,search,NaN,Hienoa! Löysit 3 tehtävään sopivaa sivua. Haku...,"{""0"":{""assignment_id"":""03d9d713-81e4-4b57-b9f0...",2023-11-02 10:51:46.016,491.552,1
7551e348-886e-4f43-be1e-2330ff3155a2,03d9d713-81e4-4b57-b9f0-80397d5e2fca,snippet_micro,NaN,Hyvä yritys! Olet valinnut tästä tekstistä 3 p...,"{""text_id"": 330, ""feedback_code"": 3, ""details""...",2023-11-02 10:54:54.665,680.201,2
3dc150f2-a981-451b-81b1-d27373ad474d,03d9d713-81e4-4b57-b9f0-80397d5e2fca,snippet_micro,NaN,Hyvin tehty! Olet valinnut 1 pääkohdan tästä t...,"{""text_id"": 339, ""feedback_code"": 1, ""details""...",2023-11-02 10:55:36.949,722.485,3
e882d7b0-7278-4823-93f6-a9518ee1aff4,03d9d713-81e4-4b57-b9f0-80397d5e2fca,snippet_micro,NaN,Hienosti yritetty! Olet valinnut tästä tekstis...,"{""text_id"": 331, ""feedback_code"": 2, ""details""...",2023-11-02 10:58:09.566,875.102,4
f3b43013-f610-4ef6-af4c-1c15742addc1,07fb2b8e-7108-4404-a70d-0e271a808471,snippet_micro,NaN,Pääkohtien valinnan tarkoituksena on löytää te...,"{""text_id"": 309, ""feedback_code"": 0, ""details""...",2023-11-07 07:34:32.494,569.394,1
...,...,...,...,...,...,...,...,...
27679427-a944-413c-a77e-3dbcf2485df8,fcd24daf-3e73-483e-bcd6-685c7e7ecae9,snippet_micro,NaN,Hienosti yritetty! Olet valinnut tästä tekstis...,"{""text_id"": 303, ""feedback_code"": 2, ""details""...",2023-11-03 08:52:27.125,1703.552,4
0bd2c753-83fa-42ec-8fc7-e7e31040e346,fddf139b-4688-4573-bec1-e772369d55bf,search,NaN,Hyvä! Olet vieraillut 2 tehtävään sopivalla si...,"{""0"":{""assignment_id"":""fddf139b-4688-4573-bec1...",2023-11-03 08:39:22.808,843.864,1
5f8d2e81-aacb-4b5a-9c24-60f56662eab1,feb14591-7d71-4fa7-ae12-21a7010d8013,search,NaN,Hyvä! Olet vieraillut 2 tehtävään sopivalla si...,"{""0"":{""assignment_id"":""feb14591-7d71-4fa7-ae12...",2023-11-02 07:45:05.915,1040.359,1


In [22]:
# Use 911e7cad-2261-4755-825d-dd734c8bfaa1 for testing
# Find the starting time of the student
print(df_assignment.loc['911e7cad-2261-4755-825d-dd734c8bfaa1','created'])
df_feedback[df_feedback['assignment_id']=='911e7cad-2261-4755-825d-dd734c8bfaa1']

2023-11-02 07:24:41.273000


,assignment_id,phase,type_id,feedback_text,details,created,time_elapsed,count
action_id,,,,,,,,
bb718593-9649-47da-b58b-585410a0beb4,911e7cad-2261-4755-825d-dd734c8bfaa1,snippet_micro,NaN,Hyvä edistys! Olet valinnut 1 pääkohdan tästä ...,"{""text_id"": 309, ""feedback_code"": 1, ""details""...",2023-11-02 07:32:03.971,442.698,1
464bcbe8-7f0f-4ea6-8002-965ed2c57289,911e7cad-2261-4755-825d-dd734c8bfaa1,snippet_micro,NaN,Hyvä edistys! Olet valinnut 1 pääkohdan tästä ...,"{""text_id"": 308, ""feedback_code"": 1, ""details""...",2023-11-02 07:38:06.485,805.212,2
38bfb2e2-5519-4d03-b559-9a6ecd5f4526,911e7cad-2261-4755-825d-dd734c8bfaa1,snippet_micro,NaN,Hienosti yritetty! Olet valinnut 4 pääkohtaa t...,"{""text_id"": 308, ""feedback_code"": 1, ""details""...",2023-11-02 07:39:21.358,880.085,3
074554b8-a002-43a6-ae2a-5cb0ee5863ec,911e7cad-2261-4755-825d-dd734c8bfaa1,snippet_micro,NaN,Hyvä yritys! Olet valinnut 2 pääkohtaa tästä t...,"{""text_id"": 308, ""feedback_code"": 1, ""details""...",2023-11-02 07:53:01.472,1700.199,4


In [23]:
# Export to Excel
if (export_feedback_table):
    # Convert 'created' column to timezone-naive datetimes as Excel does not support timezone
    df_feedback_export = df_feedback_filtered.copy()
    df_feedback_export['created'] = df_feedback_export['created'].dt.tz_localize(None)
    df_feedback_export.to_excel(outputfile_feedback)

## Analysis

In [24]:
# Check the number of each type of feedback
df_feedback_filtered.groupby('phase').describe()

type_id                              time_elapsed                \
                count mean std min 25% 50% 75% max        count          mean   
phase                                                                           
search            0.0  NaN NaN NaN NaN NaN NaN NaN        153.0  16792.515013   
snippet_micro     0.0  NaN NaN NaN NaN NaN NaN NaN        189.0   1069.655820   

               ...                         count                           \
               ...       75%          max  count      mean       std  min   
phase          ...                                                          
search         ...  1486.741  1211160.867  153.0  3.346405  3.048555  1.0   
snippet_micro  ...  1425.468     2248.365  189.0  3.603175  2.901979  1.0   

                                    
               25%  50%  75%   max  
phase                               
search         1.0  2.0  4.0  15.0  
snippet_micro  2.0  3.0  4.0  15.0  

[2 rows x 24 columns]

In [25]:
# Group by 'count' and calculate the mean of 'time_elapsed' for each group
average_time_elapsed_per_count = df_feedback_filtered.groupby('count')['time_elapsed'].describe()

# Display the average time elapsed for counts from 1 to 15
average_time_elapsed_per_count.loc[1:15]

,count,mean,std,min,25%,50%,75%,max
count,,,,,,,,
1,97.0,648.973557,465.990301,55.473,322.99100,491.5520,877.86000,2400.208
2,73.0,980.639205,483.341900,157.485,595.74900,916.9090,1282.17000,2174.635
3,55.0,1155.782891,495.803437,219.035,799.10350,1106.9780,1434.25850,2302.412
4,35.0,1195.185429,482.722849,306.003,838.42500,1154.5900,1455.31650,2248.365
5,20.0,1206.434400,381.986222,569.010,922.48975,1231.9280,1565.13225,1813.233
6,14.0,1234.372929,369.333721,631.783,987.94150,1157.7780,1414.88325,1935.007
7,12.0,1408.770083,329.841750,906.575,1241.66825,1359.0595,1565.02275,1994.038
8,10.0,1584.547400,364.514256,912.862,1423.61400,1601.9705,1896.53125,2030.647
9,5.0,1427.588600,292.318124,918.979,1463.92800,1523.1880,1586.63600,1645.212
